# Year end release cleanup

This notebook automatically moves a full year of releases into their own subdirectory under `~site/releases/` and collates the releases into their yearly listing page, including a preliminary tidy up of any links broken by moving these files. 

It's easiest to run this notebook at the start of the new year, when all the releases for the previous year are ready to be collated, thus requiring minimum adjustment from you.

After running the notebook, you can locate the new yearly release subdirectory that was inserted into our `About > Releases` section for you to clean up further, along with a live preview of the site to get you started.

## Contents
- [Prerequisites](#toc1_)    
- [Setting up](#toc2_)    
  - [Import yearly cleanup script](#toc2_1_)    
  - [Specify the year](#toc2_2_)    
  - [Retrieve matching release folders](#toc2_3_)    
- [Creating the yearly folder](#toc3_)    
  - [Create the yearly folder](#toc3_1_)    
  - [Move releases into yearly folder](#toc3_2_)    
  - [Create yearly listing page](#toc3_3_)    
  - [Edit the yearly listing page](#toc3_4_)    
  - [Retrieve moved releases](#toc3_5_)    
  - [Add moved releases to listing page](#toc3_6_)    
- [Updating sidebar and links](#toc4_)    
  - [Add yearly release folder to sidebar](#toc4_1_)    
  - [Move year end marker](#toc4_2_)    
  - [Fix broken filepaths](#toc4_3_)    
  - [Retrieve relative paths](#toc4_4_)    
- [Next steps](#toc5_)    
  - [Show files to commit](#toc5_1_)    
  - [Preview changes](#toc5_2_)    
  - [Wrap it up](#toc5_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1_'></a>

## Prerequisites 

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>Refer our release notes guide for the full steps needed before running this notebook:</b></span>
<br></br>
<a href="https://www.notion.so/validmind/Create-Release-Notes-c4291bb92f644c5eb490cc21a8a79ca5?pvs=4#0fd19127eb128090b9fde4965384be5a" style="color: #DE257E;"><b>Create Release Notes — Yearly release cleanup</b></a></div>

This process assumes you have knowledge of how to work with GitHub, including managing release tags, sending commits, and creating pull requests. 

- [ ] ALL releases from the previous year should be ready to be collated at the beginning of the new year.
- [ ] You should be on a separate branch associated to the Story for the release notes cleanup before you begin.

<a id='toc2_'></a>

## Setting up 

<a id='toc2_1_'></a>

### Import yearly cleanup script 

This cell imports our custom `yearly_cleanup.py` script for use with a shorted alias, including any necessary dependencies.

In [ ]:
import yearly_cleanup as yc

<a id='toc2_2_'></a>

### Specify the year 

You'll be asked to enter the year of releases to run the cleanup script on.

**Leave blank for the default year, which is set to the previous year.**

In [ ]:
year = yc.get_year()

<a id='toc2_3_'></a>

### Retrieve matching release folders 

This retrieves a list of release folders in `~site/releases/` that match your specified year.

**The list below will return the files sorted alphabetically for your review, like how they display in the file tree.**

In [ ]:
yc.get_yearly_releases(year)

<a id='toc3_'></a>

## Creating the yearly folder 

<a id='toc3_1_'></a>

### Create the yearly folder 

The following cell creates a new subdirectory in `~site/releases/` based on your specified year.

In [ ]:
yearly_path = yc.create_year_folder(year)

<a id='toc3_2_'></a>

### Move releases into yearly folder 

Once we have the folder available, your matching release folders will get moved into this new yearly subdirectory.

In [ ]:
yc.move_yearly_releases(yearly_path, yc.release_folders)

<a id='toc3_3_'></a>

### Create yearly listing page 

This cell copies the template from `~internal/templates/yearly-releases.qmd` and slots it into the new yearly folder as `{year}-releases.qmd` so we can begin building the yearly listings.

In [ ]:
yearly_release = yc.copy_template(yearly_path, year)

<a id='toc3_4_'></a>

### Edit the yearly listing page 

Before we can add the listings for the moved releases to the new yearly page, we need to make sure the following has the right year:

- The title
- The listing ID
- The listing embed itself

This cell takes care of all of the above for us.

In [ ]:
if yearly_release:
    yc.update_template(yearly_release, year)

<a id='toc3_5_'></a>

### Retrieve moved releases 

This cell returns the `release-notes.qmd` filepaths for all the release folders we just moved into the yearly subdirectory.

In [ ]:
release_listings = yc.get_release_listings(yearly_path)

<a id='toc3_6_'></a>

### Add moved releases to listing page 

Next, we'll insert the moved release files into the listing for the yearly roundup page sorted by the release dates in descending order (newest first).

In [ ]:
if release_listings:
    yc.update_listing(yearly_release, release_listings)

<a id='toc4_'></a>

## Updating sidebar and links 

<a id='toc4_1_'></a>

### Add yearly release folder to sidebar 

Since we moved our releases for the specified year into their own subfolder, we'll need to update the sidebar in `_quarto.yml` to accomodate.

**This cell takes all the release filepaths we just moved into our yearly folder and shoves them into a `contents:` accordion menu with the new `{year}-.releases.qmd` listing page as the landing page.** 


In [ ]:
yc.update_quarto_yaml(year)

<a id='toc4_2_'></a>

### Move year end marker 

This cell moves our marker for where the current year's releases end to the top above our yearly accordion menu so that we are ready for the new year.

In [ ]:
yc.move_year_marker()

<a id='toc4_3_'></a>

### Fix broken filepaths 

This cell looks for absolute filepaths in `.qmd` and `.yml` files in `~site/` matching `releases/{year}-` and renames them `releases/{year}/{year}-` to accomodate for the releases we moved.

In [ ]:
yc.update_paths(year)

<a id='toc4_4_'></a>

### Retrieve relative paths 

This cell looks for relative paths (`../example.qmd`) in the `~site/releases/` folder that might need manual adjustment, such as in listings or any links that don't follow our proper `/root` convention but should. 

**You will ned to review these links and edit them if necessary to ensure that the filepaths are not broken after the move.**

In [ ]:
yc.search_links(yearly_path)

<a id='toc5_'></a>

## Next steps 

<a id='toc5_1_'></a>

### Show files to commit 

This block shows you the files changed or added you need to commit — it should omit any files in `~release-scripts/`. 

In [ ]:
%%bash
git status | grep -v 'release-scripts/'

<a id='toc5_2_'></a>

### Preview changes 

Run this cell to preview the site, including the new `{year}-releases.qmd` listing file you just created. 

In [ ]:
%%bash
cd ../site
quarto preview

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>When you're done with the preview, please restart the kernel.</b></span>
<br></br>
Make sure that when you commit your changes to remote, do NOT include the changes in the <code>~/release-scripts/</code> folder as that would commit changed notebook with the output cells.</div>

<a id='toc5_3_'></a>

### Wrap it up 

**You may want to send a commit up to remote before you begin editing so you have a backup.**

- [ ] Make sure that the new yearly accordion menu displays as expected in the sidebar under `About > Releases`.
- [ ] Double-check the files edited by `Fix broken filepaths` to ensure that the new links resolve correctly.
- [ ] Resolve any relative links found in `Retrieve relative paths` that were broken by the move. 
- [ ] Review the summaries at the top of each release page to make sure they look good on the listing tiles.